In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle as pkl

In [6]:
with open('/workspaces/codespaces-jupyter/data/btcusd_15min.pkl', 'rb') as f:
    df = pkl.load(f)
df.head()

,Timestamp,open,close,high,low,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325413e+09,4.58,4.58,4.58,4.58,0.0
2,1.325414e+09,4.58,4.58,4.58,4.58,0.0
3,1.325415e+09,4.58,4.58,4.58,4.58,0.0
4,1.325416e+09,4.58,4.58,4.58,4.58,0.0


In [7]:
df.shape

(450018, 6)

In [ ]:
def calculate_macd(df, fast=12, slow=26, signal=9):
    df['EMA_fast'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['Signal_Line'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
    return df


In [ ]:
def bolinger_bands(df, window=20, std=2):
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['BB_up'] = df['SMA'] + (df['Close'].rolling(window=window).std() * std)
    df['BB_down'] = df['SMA'] - (df['Close'].rolling(window=window).std() * std)
    return df

In [ ]:
def momentum_osilator(df, window=10):
    df['pmo'] = (df['close']/df['close'].shift(window) - 1) * 100
    return df

In [ ]:
def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


In [ ]:
def on_balance_volume(df):
    df['OBV'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], np.where(df['Close'] < df['Close'].shift(1), -df['Volume'], 0)).cumsum()
    return df

### since price a has changed multiple folds, volume has also decreaed , so . . 

In [ ]:
def comodiity_channel_index(df, window=14):
    df['CCI'] = (df['Close'] - df['Close'].rolling(window=window).mean()) / (0.015 * df['Close'].rolling(window=window).std())
    return df